In [1]:
from qiskit import QuantumCircuit, execute, Aer

from pairwise_tomography.pairwise_state_tomography_circuits import pairwise_state_tomography_circuits
from pairwise_tomography.pairwise_fitter import PairwiseStateTomographyFitter
from pairwise_tomography.utils import concurrence
from pairwise_tomography.visualization import draw_entanglement_graph

from math import pi, cos, sin

In [129]:
class ParameterGraph ():
    
    def __init__ (self,num_qubits,coupling_map=[]):
        
        self.num_qubits = num_qubits
        
        self.expect = {}
        for j in range(self.num_qubits):
            self.expect[j] = {'X':0.0,'Y':0.0,'Z':1.0}
        
        self.coupling_map = []
        for j in range(self.num_qubits-1):
            for k in range(j+1,self.num_qubits):
                if ([j,k] in coupling_map) or ([j,k] in coupling_map) or (not coupling_map):
                    self.coupling_map.append([j,k])
                    self.expect[j,k] = {'XX':0.0,'XY':0.0,'XZ':0.0,
                                        'YX':0.0,'YY':0.0,'YZ':0.0,
                                        'ZX':0.0,'ZY':0.0,'ZZ':1.0}
                    
        self.qc = QuantumCircuit(self.num_qubits)
        
    def _update_expect(self):
        
        tomo_circs = pairwise_state_tomography_circuits(self.qc, self.qc.qregs[0])
        job = execute(tomo_circs, Aer.get_backend('qasm_simulator'), shots=8192)
        raw_expect = PairwiseStateTomographyFitter(job.result(), tomo_circs, self.qc.qregs[0]).fit(output='expectation')
        
        self.expect = {}
        for j in range(self.num_qubits):
            self.expect[j] = {'X':0.0,'Y':0.0,'Z':0.0}
        degree = {j:0 for j in range(self.num_qubits)}
        for pair in self.coupling_map:
            (x,y) = tuple(pair)
            self.expect[x,y] = {}
            for paulis in raw_expect[x,y]:
                if 'I' not in paulis:
                    self.expect[x,y][paulis[0]+paulis[1]] = raw_expect[x,y][paulis]
                else:
                    pauli = list(paulis)
                    pauli.remove('I')
                    pauli = pauli[0]
                    k = paulis.index(pauli)
                    self.expect[pair[k]][pauli] += raw_expect[tuple(pair)][paulis]
                    degree[pair[k]] += 1/3
        for j in range(self.num_qubits):
            for pauli in ['X','Y','Z']:
                self.expect[j][pauli] = self.expect[j][pauli]/degree[j]
        
    def transfer(self,params,_control_params=None):
        '''
            Implements a transfer between single qubit quantities, as described by `params`.
            This should take the form:
                * params[0] - The primary quantity whose value is being changed.
                * params[1] - The quantity whose value will change to compensate.
                * params[2] - The fraction of a flip by which the primary value is changed.
                * params[3] - The qubit on which the transfer is applied.
            The `_control_params` kwarg details on the , see `controlled_transfer()`.
        '''
        
        def basis_change(basis,qubit,dagger=False):
            '''
                Returns the circuit required to change to the Z basis from the eigenbasis
                of a particular Pauli. The opposite is done when `dagger=True`.
            '''
            basis_change = QuantumCircuit(self.num_qubits)
            if basis=='X':
                basis_change.h(0)
            elif basis=='Y':
                if dagger:
                    basis_change.rx(pi/2,0)
                else:
                    basis_change.rx(-pi/2,0)
            return basis_change
        
        sink_pauli,source_pauli,fraction,qubit = params
        paulis = [source_pauli,sink_pauli]
        
        if _control_params:
            control_pauli,control_qubit = _control_params[0],_control_params[1]
        
        theta = fraction*pi
        
        if 'Z' in paulis:
            if 'X' in paulis:
                final_Z = self.expect[qubit]['Z'] * cos(theta) + self.expect[qubit]['X'] * sin(theta)
            elif 'Y' in paulis:
                final_Z = self.expect[qubit]['Z'] # DO!
        else:
            final_Z = self.expect[qubit]['Z'] # DO!
            
        if (final_Z<self.expect[qubit]['Z'] and fraction>0) or (final_Z>self.expect[qubit]['Z'] and fraction<0):
            theta = -theta
            
        if _control_params:
            self.qc += basis_change(control_pauli,control_qubit,dagger=True)
        
        if 'X' in paulis:
            if 'Y' in paulis:
                if _control_params:
                    self.qc.crz(theta,control_qubit,qubit)
                else:
                    self.qc.rz(theta,qubit)
            elif 'Z' in paulis:
                if _control_params:
                    self.qc.cry(theta,control_qubit,qubit)
                else:
                    self.qc.ry(theta,qubit)
        else:
            if _control_params:
                self.qc.mcrx(theta,[self.qc.qregs[0][control_qubit]],self.qc.qregs[0][qubit])
            else:
                self.qc.rx(pi*epsilon,qubit)
        
        if _control_params:
            self.qc += basis_change(control_pauli,control_qubit,dagger=False)
         
        if not _control_params:
            self._update_expect()
            
    
    def controlled_transfer (self,params,control_params):
        '''
            Implements a controlled transfer between single qubit quantities.
            The single qubit operation is described by `params` which should
            take the same form as for `transfer()`. The control is described
            by `control_params`
            
            
            This should take the form:
                * params[0] - The primary quantity whose value is being changed.
                * params[1] - The quantity whose value will change to compensate.
                * params[2] - The fraction of a flip by which the primary value is changed.
                * params[3] - The qubit on which the transfer is applied.
        '''
        
        def flip(pole,control_pauli,control_qubit):
            ''''''
            if pole=='-':
                if control_pauli in ['Z','Y']:
                    self.qc.x(control_qubit)
                else:
                    self.qc.z(control_qubit)
        
        pole,control_pauli,control_qubit = control_params[0][0],control_params[0][1],control_params[1]
        
        flip(pole,control_pauli,control_qubit)
        self.transfer(params,_control_params=[control_pauli,control_qubit])
        flip(pole,control_pauli,control_qubit)
        
        self._update_expect()
            
    def swap (self,params):
        ''''''
        
        qubit1,qubit2,fraction = params
        
        self.qc.cx(qubit1,qubit2)
        self.qc.crx(pi*fraction,qubit2,qubit1)
        self.qc.cx(qubit1,qubit2)
        
        _update_expect()
        
    def print_state(self,dp=3):
        
        for j in range(self.num_qubits):
            print('\nState of qubit',j)
            for pauli in ['X','Y','Z']:
                print('   ',pauli+':',round(self.expect[j][pauli],dp))

        for pair in self.coupling_map:
            (j,k) = tuple(pair)
            if (j,k) in self.expect:
                print('\nRelationship for qubits',j,'and',k)
                for paulis in self.expect[j,k]:
                    print('   ',paulis+':',round(self.expect[j,k][paulis],dp))

In [130]:
n = 3

network = ParameterGraph(n)

network.transfer(['Z','X',0.5,0])
network.transfer(['Z','X',0.25,0])
network.controlled_transfer(['Z','Y',0.5,1],['-Z',0])

network.print_state(dp=1)


State of qubit 0
    X: -0.5
    Y: -0.0
    Z: -0.7

State of qubit 1
    X: 0.0
    Y: -0.2
    Z: 0.8

State of qubit 2
    X: -0.0
    Y: 0.0
    Z: 1.0

Relationship for qubits 0 and 1
    XX: -0.0
    XY: 0.5
    XZ: -0.5
    YX: -0.5
    YY: -0.0
    YZ: -0.0
    ZX: 0.0
    ZY: -0.1
    ZZ: -0.9

Relationship for qubits 0 and 2
    XX: 0.0
    XY: -0.0
    XZ: -0.5
    YX: -0.0
    YY: 0.0
    YZ: -0.0
    ZX: 0.0
    ZY: -0.0
    ZZ: -0.7

Relationship for qubits 1 and 2
    XX: 0.0
    XY: -0.0
    XZ: 0.0
    YX: -0.0
    YY: 0.0
    YZ: -0.2
    ZX: 0.0
    ZY: 0.0
    ZZ: 0.8
